In [1]:
from copy import deepcopy
from typing import Type, Union

import pandas as pd
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit.circuit.library import ZZFeatureMap, EfficientSU2
from qiskit_machine_learning.algorithms import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit.algorithms.optimizers import L_BFGS_B
from qiskit_machine_learning.algorithms import VQC
from qiskit.circuit.library import ZFeatureMap, TwoLocal
from sklearn.linear_model import LogisticRegression


In [2]:
iris = datasets.load_iris()


def get_dataset(start, stop, columns):
    x_to_return = pd.DataFrame(iris.data[start:stop])
    y_to_return = pd.Series(iris.target[start:stop])

    x_to_return.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

    x_to_return = x_to_return.loc[:, columns]

    x_to_return = StandardScaler().fit_transform(x_to_return)

    return *train_test_split(
        x_to_return, y_to_return, test_size=0.3, random_state=1, stratify=y_to_return), x_to_return, y_to_return

# Easy separable data

## Classical approach on easy separable data

In [3]:
experiment_columns = ['petal_length', 'petal_width']
X_train, X_test, y_train, y_test, _, _ = get_dataset(0, 100, experiment_columns)

In [4]:
regressor = LogisticRegression()
regressor.fit(X_train, y_train)

regressor.score(X_test, y_test)

1.0

## Quantum approach VQC

In [5]:
# construct feature map, ansatz, and optimizer

n_of_features = X_train.shape[1]


def get_vqc_qnn(optimizer: Union[Type[ADAM], Type[L_BFGS_B], Type[COBYLA]],
                loss="cross_entropy"):
    in_feature_map = ZFeatureMap(n_of_features)

    in_ansatz = TwoLocal(2, ['ry', 'rz'], 'cz', 'full', reps=4, insert_barriers=True)

    # construct variational quantum classifier
    in_vqc = VQC(
        feature_map=in_feature_map,
        ansatz=in_ansatz,
        loss=loss,
        optimizer=optimizer(maxiter=1000)
    )

    return in_vqc

In [6]:
vqc = get_vqc_qnn(COBYLA)
vqc.fit(X_train, pd.get_dummies(y_train).to_numpy().astype(int))
vqc.score(X_test, pd.get_dummies(y_test).to_numpy().astype(int))

0.8333333333333334

In [7]:
vqc = get_vqc_qnn(ADAM)
vqc.fit(X_train, pd.get_dummies(y_train).to_numpy().astype(int))
vqc.score(X_test, pd.get_dummies(y_test).to_numpy().astype(int))

0.9

In [8]:
vqc = get_vqc_qnn(L_BFGS_B)
vqc.fit(X_train, pd.get_dummies(y_train).to_numpy().astype(int))
vqc.score(X_test, pd.get_dummies(y_test).to_numpy().astype(int))

0.9333333333333333

# Not Easy separable data

In [27]:
experiment_columns = ['petal_length', 'petal_width']
X_train_not_easy_separable, X_test_not_easy_separable, y_train_not_easy_separable, y_test_not_easy_separable, _, _ = get_dataset(
    50, 150, experiment_columns)

y_train_not_easy_separable[y_train_not_easy_separable == 2] = 0
y_test_not_easy_separable[y_test_not_easy_separable == 2] = 0

### Classical approach on not easy separable data

In [28]:
regressor = LogisticRegression()
regressor.fit(X_train_not_easy_separable, y_train_not_easy_separable)

regressor.score(X_test_not_easy_separable, y_test_not_easy_separable)

0.9666666666666667

### Quantum approach VQC

In [29]:
# construct feature map, ansatz, and optimizer

def create_vqc(in_feature_map, in_ansatz, optimizer: Union[Type[ADAM], Type[L_BFGS_B], Type[COBYLA]]):
    inner_vqc = VQC(
        feature_map=in_feature_map,
        ansatz=in_ansatz,
        loss="cross_entropy",
        optimizer=optimizer(maxiter=1000)
    )

    return inner_vqc

In [30]:
# ADAM

vqc = create_vqc(
    ZZFeatureMap(n_of_features),
    EfficientSU2(num_qubits=n_of_features, reps=2),
    optimizer=ADAM
)

vqc.fit(X_train_not_easy_separable, pd.get_dummies(y_train_not_easy_separable).to_numpy().astype(int))
vqc.score(X_test_not_easy_separable, pd.get_dummies(y_test_not_easy_separable).to_numpy().astype(int))

0.6666666666666666

In [31]:
# L_BFGS_B

vqc = create_vqc(
    ZZFeatureMap(n_of_features),
    EfficientSU2(num_qubits=n_of_features, reps=2),
    optimizer=L_BFGS_B
)

vqc.fit(X_train_not_easy_separable, pd.get_dummies(y_train_not_easy_separable).to_numpy().astype(int))
vqc.score(X_test_not_easy_separable, pd.get_dummies(y_test_not_easy_separable).to_numpy().astype(int))

0.8

In [32]:
# COBYLA

vqc = create_vqc(
    ZZFeatureMap(n_of_features),
    EfficientSU2(num_qubits=n_of_features, reps=2),
    optimizer=COBYLA
)

vqc.fit(X_train_not_easy_separable, pd.get_dummies(y_train_not_easy_separable).to_numpy().astype(int))
vqc.score(X_test_not_easy_separable, pd.get_dummies(y_test_not_easy_separable).to_numpy().astype(int))

0.6666666666666666